In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime

# Importing the Donors csv and making a few amendments to it

In [2]:
donors=pd.read_csv('Donors.csv')

In [3]:
donors['Unix Time of registeration']=donors['Unix Time of registeration'
                                           ].apply(lambda x: datetime.datetime.fromtimestamp(int(x)
                                            ).strftime('%Y-%m-%d %H:%M:%S'))

In [4]:
donors = donors.rename({'Unix Time of registeration': 'Unix Time'}, axis=1)
donors.drop(['Unnamed: 0','City'], axis=1, inplace=True)

# Importing the Order and Appointment datasets

In [5]:
orders = pd.read_csv('orders_updated_29_03.csv')
orders=orders.drop(['State'], axis = 1)
appt = pd.read_csv('Cleaned_Appointment.csv')
appt=appt.rename({'Appointment ID': 'AppID', 'Appointment Date': 'AppDate'}, axis=1)
appt = appt.drop(['Unnamed: 0'], axis = 1)

In [6]:
from datetime import datetime
orders['Date'] = orders['Unix Time'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))
orders['Year'] = orders['Unix Time'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y'))
orders['Month'] = orders['Unix Time'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%B'))
orders['Day of Week'] = orders['Unix Time'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%A'))

In [7]:
orders = orders.drop(['Unix Time','End Point Geo-location'], axis = 1)

In [8]:
orders['state_list']=orders['state_list'].replace({'Fct, Abuja': r'Abuja'}, regex=True)

# Printing the three dataframes together for comparison

In [9]:
orders.head(2)

,Order ID,Blood Type,Hospital ID,BloodBank ID,Cross Matching,QTY,Address,lat,long,Date,Year,Month,Day of Week,geometry,state_list
0,1,B+,2154730,1,Yes,5,"Military Hospital, Igbogbi, Lagos",6.522464,3.373477,2016-06-15,2016,June,Wednesday,POINT (3.3734771 6.5224642),Lagos
1,2,O+,1250873,1,Yes,1,"General Hospital Road, Araromi, Gbagada, Lagos",6.552921,3.388054,2016-06-23,2016,June,Thursday,POINT (3.3880537 6.5529208),Lagos


In [10]:
appt.head(2)

,AppID,AppDate,Time,Donor ID,Year,Month,Weekday,Hours
0,17,2016-06-18,10:00,2543,2016,6,Saturday,10
1,18,2016-06-24,09:00,2544,2016,6,Friday,9


In [11]:
donors.head(2)

,Donor_ ID,Gender,State,Blood Group,Unix Time,cities
0,1,undefined,LA,O+,2018-07-26 12:37:52,lagos
1,2,undefined,LA,unknown,2018-07-26 12:37:52,lagos


# Going to merge Appointment and Donors on Donor ID

In [12]:
appoint_donors=pd.merge(appt, donors, left_on='Donor ID',right_on='Donor_ ID',how='left')
appoint_donors.head(2)

,AppID,AppDate,Time,Donor ID,Year,Month,Weekday,Hours,Donor_ ID,Gender,State,Blood Group,Unix Time,cities
0,17,2016-06-18,10:00,2543,2016,6,Saturday,10,2543.0,male,NaN,unknown,2016-06-15 15:06:47,surulere
1,18,2016-06-24,09:00,2544,2016,6,Friday,9,2544.0,male,NaN,AB+,2016-06-16 00:25:08,unknown


# Going to use geographic information for Nigeria to populate Sates in appoint_donors dataset

import list of Nigerian geographic data from 'https://simplemaps.com/data/ng-cities'

In [13]:
nigeria=pd.read_csv('/Users/gvaccar/Downloads/ng.csv')
nigeria=nigeria.rename({'admin': 'State'}, axis=1)

Create a dictionary with cities as keys and states as vaules, which you are going to use to pupulate States in the
appoint_donors dataframe

In [14]:
keys= [x.lower() for x in nigeria['city']]
values= [x for x in nigeria['State']]
dictionary= dict(zip(keys, values))


In [15]:
appoint_donors['State']=appoint_donors['cities'].map(dictionary)
appoint_donors['State']=appoint_donors['State'].transform(lambda x: x.fillna('unknown'))

In [16]:
#cleaning the dataframe by dropping Donor_ ID 
appoint_donors=appoint_donors.drop(['Donor_ ID'], axis=1)

In [17]:
# Excluding from the dataframes the rows for which the city or state are unknown
appoint_donors['State']=appoint_donors['State'].replace({'Federal Capital Territory': r'Abuja'}, regex=True)

In [18]:
appoint_donors.head(2)

,AppID,AppDate,Time,Donor ID,Year,Month,Weekday,Hours,Gender,State,Blood Group,Unix Time,cities
0,17,2016-06-18,10:00,2543,2016,6,Saturday,10,male,Lagos,unknown,2016-06-15 15:06:47,surulere
1,18,2016-06-24,09:00,2544,2016,6,Friday,9,male,unknown,AB+,2016-06-16 00:25:08,unknown


# Stock Level Analysis

In [19]:
# makig sure that the dates for orders and appointments_donors are aligned

min_order_date= orders['Date'].min()

max_order_date= orders['Date'].max()

app_don_reduced=appoint_donors[(appoint_donors.AppDate >= min_order_date) & (
                           appoint_donors.AppDate <= max_order_date)]

In [20]:
print(app_don_reduced['AppDate'].min())
print(app_don_reduced['AppDate'].max())

2016-06-18
2020-03-16


# Creating a dataframe with daily Blood orders and daily Blood Donations

this is going to be used as a base for the calculation of the daily blood stock

In [21]:
donations=app_don_reduced[['AppDate','AppID']].groupby('AppDate').count().reset_index().copy()

In [22]:
blood_demand=orders[['Date','QTY']].groupby('Date').sum().reset_index().copy()

In [23]:
dates = pd.date_range(min_order_date, max_order_date).to_list()
df = pd.DataFrame()
df['Date'] = [datetime.strftime(i, '%Y-%m-%d') for i in dates]
df=pd.merge(df, blood_demand, on='Date', how='left')
df=pd.merge(df, donations,left_on='Date',right_on='AppDate', how='left')
df.drop('AppDate', axis=1, inplace=True)

df=df.transform(lambda x: x.fillna(0))
df.sort_values(by=['Date'], inplace=True, ascending=False)
df.reset_index(drop=True)
df.head()

,Date,QTY,AppID
1412,2020-03-16,11.0,1.0
1411,2020-03-15,28.0,1.0
1410,2020-03-14,14.0,4.0
1409,2020-03-13,22.0,0.0
1408,2020-03-12,19.0,0.0


# Implementation of the function to calcuate the stock level

In [24]:
from datetime import timedelta


#Creating a few list that we will use later on during the implementation of the function to calculate the stock
#level

df = df.sort_values('Date')

list_of_dates = pd.to_datetime(df.Date, format = "%Y-%m-%d")
list_orders = df.QTY.astype(int)
list_donations = df.AppID.astype(int)

In [25]:
def update_stock(date_list, donations, orders):
    
    Donations_stock =[0] * len(date_list)
    Tot_blood_before_using_it = [0] * len(date_list)
    Blood_stock_end_day =[0] * len(date_list)
    unfulfilled_orders=[0] * len(date_list)
    Previous_donations_stock = [0] * len(date_list)
    expired_units= [0] * len(date_list)
    
    for i in range(len(date_list)):
        
        # Add new donations to the stock based on the number of donations made that day
        # wi assume the blood donated can last for 42 days
        new_donation_expire_date = date_list[i] + timedelta(days=42)
        Add_new_donations = [new_donation_expire_date] * donations[i]
        
        if i == 0:
            Previous_donations_stock = []
            Donations_stock[i] = Add_new_donations
          
        else:
            Previous_donations_stock= Donations_stock[i -1]
            previus_plus_new = Previous_donations_stock + Add_new_donations
            
            #Here we remove the expired donations from the Donations_stock (basically we exclude from
            #Donations_stock -which is a list of dates- all the dates previous to the current date)
            Donations_stock[i] = [x for x in previus_plus_new if x >= date_list[i]]
            
            # this allows on to keep track of how many units expired
            expired_units[i] = len(previus_plus_new) - len(Donations_stock[i])
            
        #this is my total daily stock of blood that i have before using it in case of positive Orders
        Tot_blood_before_using_it[i] = len(Donations_stock[i])

        #Remove orders from Donations_stock
        if orders[i] >= len(Donations_stock[i]):
            unfulfilled_orders[i] = orders[i] - len(Donations_stock[i])
            Donations_stock[i] = []
            
        else:
            unfulfilled_orders[i] = 0
            Donations_stock[i] = Donations_stock[i][orders[i]:]
            
        #this is the blood stock that will remain at the end of the day after using it, and that will be carried
        #forward to the next day
        Blood_stock_end_day[i]= len(Donations_stock[i])
                 
    return (expired_units, Tot_blood_before_using_it,Blood_stock_end_day, unfulfilled_orders)
            
            

In [26]:
get_data=update_stock(list_of_dates, list_donations, list_orders)

In [27]:
df['Daily Blood Orders'] =  list_orders
df['Daily Blood Donations'] =  list_donations

df['Expired units'] = get_data[0]
df['Blood prev + curr day'] = get_data[1]
df['Blood stock end day'] = get_data[2]
df['Unfulfilled orders'] = get_data[3]

# renaming the columns of blood orders and donations in the previous dataframe just for the sake of clarity
df.drop(['QTY','AppID'], axis = 1, inplace=True)

df.head()

,Date,Daily Blood Orders,Daily Blood Donations,Expired units,Blood prev + curr day,Blood stock end day,Unfulfilled orders
0,2016-05-04,1,0,0,0,0,1
1,2016-05-05,0,0,0,0,0,0
2,2016-05-06,0,0,0,0,0,0
3,2016-05-07,0,0,0,0,0,0
4,2016-05-08,0,0,0,0,0,0


In [28]:
df.to_csv('blood_stock.csv', index=False)